In [140]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from math import *
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# import plotly.graph_objs as go
# import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from Data_Processing import *
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)
import shutil
%matplotlib inline
import seaborn as sns
import datetime
import scipy.stats as stats

In [141]:
train = pd.read_csv('./Data/jinnan_round1_train_20181227.csv', encoding = 'gb18030')
test  = pd.read_csv('./Data/jinnan_round1_testB_20190121.csv', encoding = 'gb18030')
sample_id=test['样本id']

# 修正

In [142]:
# 测试数据修正
train.loc[train['B14']==40, 'B14'] = train['B14'].mode()
# train.loc[1304, 'A25'] = 70

# for value in padding_list:
#     test.loc[test[column]==value,column]=test[column].mode()
test.loc[test['B14']==785.0, 'B14'] = 385.0
test.loc[test['A19']==310.0, 'A19'] = 300.0
test.loc[test['A19']==700.0, 'A19'] = 100.0
# test.loc[test['B6']==49.0, 'B6'] = 100.0
train.loc[1304, 'A25'] = 80
train['A25'] = train['A25'].astype(int)

# 异常值清洗

In [143]:
# 删除类别唯一的特征
for df in [train, test]:
    df.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

# 删除某一类别占比超过90%的列
good_cols = list(train.columns)
print('类别占比>0.9的列:(列名, 占比)')
for col in train.columns:
    rate = train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        good_cols.remove(col)
        print(col,rate)

# 暂时不删除，后面构造特征需要
good_cols.append('A1')
good_cols.append('A3')
good_cols.append('A4')

# 删除异常值
train = train[train['收率']>0.87]

train = train[good_cols]
test['收率'] = -1
test  = test[good_cols]

# 合并数据集
target = train['收率']
data = pd.concat([train,test],axis=0,ignore_index=True)
data = data.fillna(-1)

# 异常时间处理
def timeTranSecond(t):
    try:
        t,m,s=t.split(":")
    except:
        if t=='1900/1/9 7:00':
            return 7*3600/3600
        elif t=='1900/1/1 2:30':
            return (2*3600+30*60)/3600
        elif t=="700":
            return (7*3600)/3600
        elif t==-1:
            return -1
        else:
            return 0
    
    try:
        tm = (int(t)*3600+int(m)*60+int(s))/3600
    except:
        return (30*60)/3600
    return tm


for f in ['A5','A7','A9','A11','A14','A16','A24','A26','B5','B7']:
    try:
        data[f] = data[f].apply(timeTranSecond)
    except:
        print(f,'应该在前面被删除了！')

        
def getDuration(se):
    try:
        sh,sm,eh,em=re.findall(r"\d+\.?\d*",se)
    except:
        if se == -1:
            return -1 
        
    try:
        if int(sh)>int(eh):
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600 + 24
        else:
            tm = (int(eh)*3600+int(em)*60-int(sm)*60-int(sh)*3600)/3600
    except:
        if se=='19:-20:05':
            return 1
        elif se=='15:00-1600':
            return 1
    return tm


for f in ['A20','A28','B4','B9','B10','B11']:
    data[f] = data.apply(lambda df: getDuration(df[f]), axis=1)
    


类别占比>0.9的列:(列名, 占比)
A1 0.9863896848137536
A2 0.9699140401146131
A3 0.9570200573065902
A4 0.9570200573065902
B2 0.9842406876790831


# 特征定义

In [144]:
# 原始特征
orig_fea_B = ['B1', 'B5','B6','B7', 'B8', 'B12', 'B14', 'A19']
encoder_fea = []
b14_fea = []

# id＋比例＋温度＋时间    注意归一化

In [145]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

In [146]:
# 获取样本id值特征
data['样本id'] = data['样本id'].apply(lambda x: int(x.split('_')[1]))

# 原料比例特征
data['b12/b14'] = data['B12']/data['B14']
data['a19_rate0'] = data['A19']/(data['A1']+data['A3']+data['A4']+data['B14']+data['B1']+data['B12'])
data['b1_rate0'] = data['B1']/(data['A1']+data['A3']+data['A4']+data['A19']+data['B14']+data['B12'])
data['b12_rate0'] = data['B12']/(data['A1']+data['A3']+data['A4']+data['A19']+data['B1']+data['B14'])
data['b14_rate0'] = data['B14']/(data['A1']+data['A3']+data['A4']+data['A19']+data['B1']+data['B12'])

# 温度特征，归一化
data['A_tem_sum'] = data['A6']+data['A8']+data['A10']+data['A12']+data['A15']+data['A17']+data['A25']+data['A27']
data['A_tem_sum'] = data[['A_tem_sum']].apply(max_min_scaler)

# time，归一化
data['A_B_time_sum'] = data['A5']+data['A7']+data['A9']+data['A11']+data['A14']+data['A16']+data['A24']+data['A26']+data['B5']+data['B7']
data['A_B_time_sum'] = data[['A_B_time_sum']].apply(max_min_scaler)

numerical_columns = ['b12/b14', 'b14_rate0', 'a19_rate0', 'b1_rate0','b12_rate0', 'A_tem_sum','A_B_time_sum']

# B14 Click through rate+组合特征

In [147]:
# 强特B14的分组特征：对B14分组，求收率的max, min, mean
df0 = data.groupby(['B14'], as_index=False)['收率'].agg({  'gb_B14_tar'+ '_mean': 'mean', 
                                                          'gb_B14_tar'+ '_max': 'max', 
                                                          'gb_B14_tar' + '_min': 'min'})
data = pd.merge(data, df0, how='left', on=['B14'])

b14_fea.append('gb_B14_tar_mean')
b14_fea.append('gb_B14_tar_max')
b14_fea.append('gb_B14_tar_min')

# 与B14交叉特征
for feat in orig_fea_B+['样本id']:  
    if feat != 'B14':
        col_name = feat + '/B14'
        encoder_fea.append(col_name)
        data[col_name] = data[feat]/data['B14']
        data[col_name] =  data[[col_name]].apply(max_min_scaler)


# 数据编码

In [148]:
#label encoder
for f in orig_fea_B+encoder_fea:
    data[f] = data[f].map(dict(zip(data[f].unique(), range(0, data[f].nunique()))))
train = data[:train.shape[0]]
test  = data[train.shape[0]:]

# 鱼老开源特征

In [149]:
train['target'] = target
train['intTarget'] = pd.cut(train['target'], 5, labels=False)
train = pd.get_dummies(train, columns=['intTarget'])
li = ['intTarget_0.0','intTarget_1.0','intTarget_2.0','intTarget_3.0','intTarget_4.0']
mean_columns = []
for f1 in orig_fea_B:
    cate_rate = train[f1].value_counts(normalize=True, dropna=False).values[0]
    if cate_rate < 0.90:
        for f2 in li:
            col_name = 'B14_to_'+f1+"_"+f2+'_mean'
            mean_columns.append(col_name)
            order_label = train.groupby([f1])[f2].mean()
            train[col_name] = train['B14'].map(order_label)
            miss_rate = train[col_name].isnull().sum() * 100 / train[col_name].shape[0]
            if miss_rate > 0:
                train = train.drop([col_name], axis=1)
                mean_columns.remove(col_name)
            else:
                test[col_name] = test['B14'].map(order_label)     

train.drop(li+['target'], axis=1, inplace=True)


# B14 统计特征

In [150]:
# ratio ----------------------------------------------------------------------------------------------------------------
data_df = pd.concat([train, test], axis=0, ignore_index=True)

df0 = data_df[orig_fea_B]
df0['num']=1
df_new_col = pd.DataFrame()
    
for j in range(len(orig_fea_B)):
    if orig_fea_B[j] != 'B14':
        col_name = "r_clk_"+orig_fea_B[j]+"_in_"+'B14'
        b14_fea.append(col_name)
        gb0= df0.groupby(['B14',orig_fea_B[j]])['num'].sum()
        df1 = df0[['B14',orig_fea_B[j]]]
        num = df0['B14'].map(data_df['B14'].value_counts())
        df_new_col[col_name] = ((pd.merge(df1,gb0.reset_index(),how='left',on=['B14',orig_fea_B[j]]).sort_index()['num'].fillna(value=0)/num)*100).astype(int).values

for j in range(len(orig_fea_B)):
    if orig_fea_B[j] != 'B14':
        col_name = "r_clk_"+'B14'+"_in_"+orig_fea_B[j]
        b14_fea.append(col_name)
        gb0= df0.groupby([orig_fea_B[j],'B14'])['num'].sum()
        df1 = df0[[orig_fea_B[j],'B14']]
        num = df0[orig_fea_B[j]].map(data_df[orig_fea_B[j]].value_counts())
        df_new_col[col_name] = ((pd.merge(df1,gb0.reset_index(),how='left',on=[orig_fea_B[j],'B14']).sort_index()['num'].fillna(value=0)/num)*100).astype(int).values
data_df = pd.concat([data_df, df_new_col], axis=1)

df0 = data_df[orig_fea_B]
df_new_col = pd.DataFrame()
df0['num']=1
for i in range(len(orig_fea_B)):
    if orig_fea_B[i] != 'B14':
        col_name = "num_click_of_"+'B14'+"_and_"+orig_fea_B[i]
        b14_fea.append(col_name)
        gb0= df0.groupby([orig_fea_B[i],'B14'])['num'].sum()
        df1 = df0[[orig_fea_B[i],'B14']]
        gb0= (pd.merge(df1,gb0.reset_index(),how='left',
                        on=[orig_fea_B[i],'B14']).sort_index()['num'].fillna(value=0)).astype(int)
        df_new_col[col_name] = ((gb0-gb0.min())/(gb0.max()-gb0.min())*100).fillna(value=0).astype(int).values
data_df = pd.concat([data_df, df_new_col], axis=1)

# id进行分箱，求每个箱子的收率均值

In [151]:
# id对收率是正弦特点，　周期大概是５００
data_my_df_conf = data_df.copy()
id_splitN_fea = ['sign_num_id_16', 'sign_num_id_32']
def get_id_splitN(x, N):
    a = 1994/N
    interval = []
    for i in range(N):
        interval.append(a*(i+1))
    interval[-1] = 1994
    for i in range(N):
        if x<=interval[0]:
            sign_num = 1
        elif (x>interval[i]) and (x<=interval[i+1]):
            sign_num = i+2

    return sign_num

data_my_df_conf['sign_num_id_16'] = data_my_df_conf.样本id.apply(lambda x: get_id_splitN(x, 16)) #把id分成１６份
data_my_df_conf['sign_num_id_32'] = data_my_df_conf.样本id.apply(lambda x: get_id_splitN(x, 32))
train_my_df_conf = data_my_df_conf[:train.shape[0]]
test_my_df_conf  = data_my_df_conf[train.shape[0]:]

train_my_df_conf['target'] = target.values

df0 = train_my_df_conf.groupby('sign_num_id_16')['target'].mean()
data_my_df_conf['sign_num_id_16_gb_mean'] = data_my_df_conf['sign_num_id_16'].map(df0)

df0 = train_my_df_conf.groupby('sign_num_id_32')['target'].mean()
data_my_df_conf['sign_num_id_32_gb_mean'] = data_my_df_conf['sign_num_id_32'].map(df0)

del train_my_df_conf['target']

# onehot编码特征

In [152]:
data_df = data_df.fillna(-1)

train = data_df[:train.shape[0]]
test = data_df[train.shape[0]:]

X_train_df = train[['样本id'] + numerical_columns + mean_columns + b14_fea]       
X_test1 = test[['样本id'] + numerical_columns +mean_columns + b14_fea]

X_train = X_train_df.values
X_test = X_test1.values

# one hot
enc = OneHotEncoder()
for f in orig_fea_B+encoder_fea:
    enc.fit(data[f].values.reshape(-1, 1))
    X_train = sparse.hstack((X_train, enc.transform(train[f].values.reshape(-1, 1))), 'csr')
    X_test = sparse.hstack((X_test, enc.transform(test[f].values.reshape(-1, 1))), 'csr')
print(X_train.shape)
print(X_test.shape)

(1381, 2445)
(149, 2445)


In [153]:
y_train = target.values

In [154]:
# train['out_lines'] = train['收率'].apply(lambda x: 1 if x<=0.87 or  x>=0.999 else  0)
# train['out_lines'].value_counts()

In [155]:
param = {'num_leaves': 48,
         'min_data_in_leaf': 10,
         'objective': 'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=8, shuffle=True, random_state=2018)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(X_train[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(X_train[val_idx], y_train[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(X_train[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, target)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[500]	training's l2: 9.18774e-05	valid_1's l2: 0.000106459
[1000]	training's l2: 7.4917e-05	valid_1's l2: 9.94349e-05
[1500]	training's l2: 6.87251e-05	valid_1's l2: 9.73198e-05
Early stopping, best iteration is:
[1744]	training's l2: 6.67251e-05	valid_1's l2: 9.69594e-05
fold n°2
Training until validation scores don't improve for 100 rounds.
[500]	training's l2: 8.79923e-05	valid_1's l2: 0.00013538
[1000]	training's l2: 7.1453e-05	valid_1's l2: 0.000128412
[1500]	training's l2: 6.61832e-05	valid_1's l2: 0.000127699
Early stopping, best iteration is:
[1422]	training's l2: 6.62264e-05	valid_1's l2: 0.000127688
fold n°3
Training until validation scores don't improve for 100 rounds.
[500]	training's l2: 8.66078e-05	valid_1's l2: 0.000139037
Early stopping, best iteration is:
[638]	training's l2: 7.86834e-05	valid_1's l2: 0.000138386
fold n°4
Training until validation scores don't improve for 100 rounds.
[500]	training

In [156]:
##### xgb
xgb_params = {'eta': 0.005, 
              'max_depth': 10, 
              'subsample': 0.8, 
              'colsample_bytree': 0.8, 
              'objective': 'reg:linear', 
              'eval_metric': 'rmse', 
              'silent': True, 
              }

folds = KFold(n_splits=8, shuffle=True, random_state=2018)
oof_xgb = np.zeros(len(train))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X_train[trn_idx], y_train[trn_idx])
    val_data = xgb.DMatrix(X_train[val_idx], y_train[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=1000, verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X_train[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(X_test), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, target)))

fold n°1
[0]	train-rmse:0.423257	valid_data-rmse:0.422079
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 1000 rounds.
[100]	train-rmse:0.257176	valid_data-rmse:0.255865
[200]	train-rmse:0.156578	valid_data-rmse:0.155614
[300]	train-rmse:0.095704	valid_data-rmse:0.095077
[400]	train-rmse:0.058987	valid_data-rmse:0.058577
[500]	train-rmse:0.036892	valid_data-rmse:0.03689
[600]	train-rmse:0.023692	valid_data-rmse:0.024227
[700]	train-rmse:0.015917	valid_data-rmse:0.017169
[800]	train-rmse:0.011503	valid_data-rmse:0.013573
[900]	train-rmse:0.009149	valid_data-rmse:0.011927
[1000]	train-rmse:0.007907	valid_data-rmse:0.011194
[1100]	train-rmse:0.007218	valid_data-rmse:0.010855
[1200]	train-rmse:0.00675	valid_data-rmse:0.010667
[1300]	train-rmse:0.006393	valid_data-rmse:0.010555
[1400]	train-rmse:0.006073	valid_data-rmse:0.010447
[1500]	train-rmse:0.005766	valid_data-rmse:0.010374
[1600]	train-rms

[1600]	train-rmse:0.005235	valid_data-rmse:0.011725
[1700]	train-rmse:0.005023	valid_data-rmse:0.011731
[1800]	train-rmse:0.004837	valid_data-rmse:0.011737
[1900]	train-rmse:0.004655	valid_data-rmse:0.011731
[2000]	train-rmse:0.004481	valid_data-rmse:0.011728
[2100]	train-rmse:0.004307	valid_data-rmse:0.011731
Stopping. Best iteration:
[1121]	train-rmse:0.006706	valid_data-rmse:0.011667

fold n°4
[0]	train-rmse:0.422642	valid_data-rmse:0.42639
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 1000 rounds.
[100]	train-rmse:0.256799	valid_data-rmse:0.260486
[200]	train-rmse:0.156351	valid_data-rmse:0.159701
[300]	train-rmse:0.095563	valid_data-rmse:0.098563
[400]	train-rmse:0.058903	valid_data-rmse:0.061442
[500]	train-rmse:0.036855	valid_data-rmse:0.039052
[600]	train-rmse:0.02367	valid_data-rmse:0.025817
[700]	train-rmse:0.015885	valid_data-rmse:0.018395
[800]	train-rmse:0.011419	valid_data-rm

[200]	train-rmse:0.156601	valid_data-rmse:0.155275
[300]	train-rmse:0.095718	valid_data-rmse:0.094558
[400]	train-rmse:0.058999	valid_data-rmse:0.058064
[500]	train-rmse:0.036938	valid_data-rmse:0.036364
[600]	train-rmse:0.023719	valid_data-rmse:0.023711
[700]	train-rmse:0.015906	valid_data-rmse:0.016737
[800]	train-rmse:0.011484	valid_data-rmse:0.013248
[900]	train-rmse:0.009109	valid_data-rmse:0.01166
[1000]	train-rmse:0.007884	valid_data-rmse:0.010988
[1100]	train-rmse:0.007209	valid_data-rmse:0.01071
[1200]	train-rmse:0.006756	valid_data-rmse:0.010575
[1300]	train-rmse:0.006367	valid_data-rmse:0.010492
[1400]	train-rmse:0.00603	valid_data-rmse:0.010437
[1500]	train-rmse:0.005711	valid_data-rmse:0.010405
[1600]	train-rmse:0.005477	valid_data-rmse:0.010386
[1700]	train-rmse:0.005238	valid_data-rmse:0.010375
[1800]	train-rmse:0.005027	valid_data-rmse:0.01037
[1900]	train-rmse:0.004834	valid_data-rmse:0.010367
[2000]	train-rmse:0.004663	valid_data-rmse:0.010355
[2100]	train-rmse:0.0044

In [157]:
# 将lgb和xgb的结果进行stacking
train_stack = np.vstack([oof_lgb,oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=8, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,target)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 16
    
mean_squared_error(target.values, oof_stack)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9
fold 10
fold 11
fold 12
fold 13
fold 14
fold 15


0.00011900117127429142

In [158]:
sub_df=pd.DataFrame()
sub_df[0]=sample_id
sub_df[1]=predictions
sub_df[1]=sub_df[1].apply(lambda x:round(x,3))
sub_df.to_csv("./submit_jinnan_10.csv", index=False, header=None)

In [24]:
# 高级本地存储操作，，，
mk_new_dir = False
if mk_new_dir==True:
    sub_df = pd.read_csv('../jinnan_round1_submit_20181227.csv', header=None)
    sub_df[1] = predictions
    sub_df[1] = sub_df[1].apply(lambda x:round(x, 3))

    time_str = datetime.datetime.now().strftime('%m_%d_%H_%M')
    new_dir = '../submit/{}_jinnan/'.format(time_str)
    os.mkdir(new_dir)
    sub_df.to_csv(new_dir+"submit_{}.csv".format(time_str), index=False, header=None)
    shutil.copyfile('jinnan.ipynb', '../submit/{}_jinnan/{}.ipynb'.format(time_str, time_str))